In [2]:
from pathlib import Path

from jax.random import PRNGKey

from jax_unirep import evotune
from jax_unirep.evotuning_models import mlstm64
from jax_unirep.utils import dump_params

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [6]:
# Test sequences:
sequences = ["HASTA", "VISTA", "ALAVA", "LIMED", "HAST", "HAS", "HASVASTA"] * 5
holdout_sequences = [
    "HASTA",
    "VISTA",
    "ALAVA",
    "LIMED",
    "HAST",
    "HASVALTA",
] * 5
PROJECT_NAME = "evotuning_temp"


HASTA
VISTA
ALAVA
LIMED
HAST
HAS
HASVASTA
HASTA
VISTA
ALAVA
LIMED
HAST
HAS
HASVASTA
HASTA
VISTA
ALAVA
LIMED
HAST
HAS
HASVASTA
HASTA
VISTA
ALAVA
LIMED
HAST
HAS
HASVASTA
HASTA
VISTA
ALAVA
LIMED
HAST
HAS
HASVASTA


In [4]:
init_fun, apply_fun = mlstm64()

# The input_shape is always going to be (-1, 26),
# because that is the number of unique AA, one-hot encoded.
_, inital_params = init_fun(PRNGKey(42), input_shape=(-1, 26))

# 1. Evotuning with Optuna
n_epochs_config = {"low": 1, "high": 1}
lr_config = {"low": 1e-5, "high": 1e-3}
study, evotuned_params = evotune(
    sequences=sequences,
    model_func=apply_fun,
    params=inital_params,
    out_dom_seqs=holdout_sequences,
    n_trials=2,
    n_splits=2,
    n_epochs_config=n_epochs_config,
    learning_rate_config=lr_config,
)

[I 2023-02-02 06:41:29,279] A new study created in memory with name: no-name-06c7a487-74d2-4f63-990a-e6db5932a618
/home/ubuntu/Maverick/evotuning/jax_unirep/evotuning.py:464: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  n_epochs = trial.suggest_discrete_uniform(**n_epochs_kwargs)
/home/ubuntu/Maverick/evotuning/jax_unirep/evotuning.py:465: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform(**learning_rate_kwargs)


right-padding sequences:   0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:evotuning:Calculations for training set:
INFO:evotuning:Epoch 0: Estimated average loss: 0.15365229547023773. 
[W 2023-02-02 06:41:42,182] Trial 0 failed with parameters: {'n_epochs': 1.0, 'learning_rate': 0.0006553116313085738} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/tensorflow/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/home/ubuntu/Maverick/evotuning/jax_unirep/evotuning.py", line 600, in objective_func
    return objective(
  File "/home/ubuntu/Maverick/evotuning/jax_unirep/evotuning.py", line 481, in objective
    evotuned_params = fit(
  File "/home/ubuntu/Maverick/evotuning/jax_unirep/evotuning.py", line 373, in fit
    state = step(i, x, y, state)
  File "/opt/tensorflow/lib/python3.10/site-packages/jax/_src/traceback_util.py", line 162, in reraise_with_filtered_traceback
    return fun(*args, **kwargs)
  File "/opt/tensorflow/lib/pyth

KeyboardInterrupt: 

In [ ]:
dump_params(evotuned_params, Path(PROJECT_NAME))
print("Evotuning done! Find output weights in", PROJECT_NAME)
print(study.trials_dataframe())